In [1]:
import pandas as pd
uid_train = pd.read_csv('../JDATA_train/uid_train.txt',delimiter='\t',header=None)
uid_train.columns = ['uid','label']

sms = pd.read_csv('../JDATA_train/sms_train.txt',delimiter='\t',header=None)
sms.columns = ['uid','opp_num','opp_head','opp_len','start_time','in_out']

voice = pd.read_csv('../JDATA_train/voice_train.txt',delimiter='\t',header=None)
voice.columns = ['uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out']

wa = pd.read_csv('../JDATA_train/wa_train.txt',delimiter='\t',header=None)
wa.columns = ['uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date']


/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
print(uid_train.shape)
uid_train.head(3)


(4999, 2)


,uid,label
0,u0001,0
1,u0002,0
2,u0003,0


In [3]:
sms.head(3)
voice.head(3)
wa.head(3)

,uid,wa_name,visit_cnt,visit_dura,up_flow,down_flow,wa_type,date
0,u0001,155导航,5.0,207.0,313.0,457.0,0.0,14.0
1,u0001,155导航,7.0,396.0,547.0,659.0,0.0,4.0
2,u0001,155导航,10.0,3212.0,781.0,941.0,0.0,12.0


In [4]:
sms.head(3)

,uid,opp_num,opp_head,opp_len,start_time,in_out
0,u4003,B378E065731B897E7295926B27CBA0D5,186,11,20174042,1
1,u4003,B378E065731B897E7295926B27CBA0D5,186,11,20174130,1
2,u4003,1B15607F3E6D167B44D46046D5993D87,189,11,20015746,0


In [5]:
voice.head(3)

,uid,opp_num,opp_head,opp_len,start_time,end_time,call_type,in_out
0,u0113,38D54642A237A11BB18455FC1E505292,132,11,26115956,26120033,1,1
1,u0113,38D54642A237A11BB18455FC1E505292,132,11,26115623,26115707,1,1
2,u0113,38D54642A237A11BB18455FC1E505292,132,11,26174233,26174321,1,1


In [6]:
sms_feat = pd.DataFrame()
grouped = sms.groupby('uid')['opp_num']
grouped

In [7]:
grouped

In [8]:
x = grouped.apply(lambda x : x.count())
x

uid
u0001     95
u0002      1
u0003     28
u0004    124
u0005     48
u0006     30
u0007     42
u0008    379
u0009     15
u0010     34
u0011      5
u0012    187
u0013      2
u0014     95
u0015     30
u0016     89
u0017     55
u0018     42
u0019    106
u0020     76
u0021      2
u0022     39
u0023      2
u0024    184
u0025    203
u0026    134
u0027    121
u0028     96
u0029     53
u0030     49
        ... 
u4970     62
u4971     30
u4972     22
u4973    137
u4974     17
u4975     36
u4976     21
u4977     25
u4978      1
u4979      7
u4980     61
u4981      1
u4982     30
u4983     14
u4984     19
u4985      8
u4986     30
u4987      7
u4988      4
u4989      4
u4990     30
u4991     31
u4992     18
u4993      7
u4994      3
u4995      6
u4996      2
u4997      3
u4998      3
u4999      6
Name: opp_num, Length: 4959, dtype: int64

In [9]:
sms_feat['uid'] = x.index
sms_feat['sms_opp_count_all'] = x.values

In [10]:
grouped = sms.groupby('uid')['opp_num']
x = grouped.apply(lambda x: len(set(x)))
sms_feat['sms_opp_count_unique'] = x.values

In [11]:
grouped = sms.groupby(['uid','in_out'])['opp_num']
x = grouped.apply(lambda x : x.count())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
sms_feat['sms_opp_count_out'] = x['0']
sms_feat['sms_opp_count_in'] = x['1']
sms_feat.head(5)

,uid,sms_opp_count_all,sms_opp_count_unique,sms_opp_count_out,sms_opp_count_in
0,u0001,95,11,39,56
1,u0002,1,1,0,1
2,u0003,28,2,0,28
3,u0004,124,10,19,105
4,u0005,48,4,2,46


In [12]:
uid_train = uid_train.merge(sms_feat,on='uid',how='left').reset_index(drop=True)
uid_train = uid_train.fillna(0)
uid_train.head(5)

,uid,label,sms_opp_count_all,sms_opp_count_unique,sms_opp_count_out,sms_opp_count_in
0,u0001,0,95.0,11.0,39.0,56.0
1,u0002,0,1.0,1.0,0.0,1.0
2,u0003,0,28.0,2.0,0.0,28.0
3,u0004,0,124.0,10.0,19.0,105.0
4,u0005,0,48.0,4.0,2.0,46.0


In [13]:
voice_feat = pd.DataFrame()
grouped = voice.groupby('uid')['opp_num']
x = grouped.apply(lambda x : x.count())
voice_feat['uid'] = x.index
voice_feat['voice_opp_count_all'] = x.values

grouped = voice.groupby('uid')['opp_num']
x = grouped.apply(lambda x : len(set(x)))
voice_feat['voice_opp_count_unique'] = x.values

grouped = voice.groupby(['uid','in_out'])['opp_num']
x = grouped.apply(lambda x : x.count())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
voice_feat['voice_opp_count_out'] = x['0']
voice_feat['voice_opp_count_in'] = x['1']

grouped = voice.groupby(['uid','call_type'])['opp_num']
x = grouped.apply(lambda x : x.count())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['1','2','3','4','5']
voice_feat['voice_count_type1'] = x['1']
voice_feat['voice_count_type2'] = x['2']
voice_feat['voice_count_type3'] = x['3']
voice_feat['voice_count_type4'] = x['4']
voice_feat['voice_count_type5'] = x['5']

voice_feat.head(5)

,uid,voice_opp_count_all,voice_opp_count_unique,voice_opp_count_out,voice_opp_count_in,voice_count_type1,voice_count_type2,voice_count_type3,voice_count_type4,voice_count_type5
0,u0001,79,22,33,46,79,0,0,0,0
1,u0002,2,2,2,0,2,0,0,0,0
2,u0003,21,15,7,14,21,0,0,0,0
3,u0004,254,77,133,121,218,6,30,0,0
4,u0005,401,55,177,224,398,0,3,0,0


In [14]:
uid_train = uid_train.merge(voice_feat,on='uid',how='left').reset_index(drop=True)
uid_train = uid_train.fillna(0)
uid_train.head(5)

,uid,label,sms_opp_count_all,sms_opp_count_unique,sms_opp_count_out,sms_opp_count_in,voice_opp_count_all,voice_opp_count_unique,voice_opp_count_out,voice_opp_count_in,voice_count_type1,voice_count_type2,voice_count_type3,voice_count_type4,voice_count_type5
0,u0001,0,95.0,11.0,39.0,56.0,79.0,22.0,33.0,46.0,79.0,0.0,0.0,0.0,0.0
1,u0002,0,1.0,1.0,0.0,1.0,2.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0
2,u0003,0,28.0,2.0,0.0,28.0,21.0,15.0,7.0,14.0,21.0,0.0,0.0,0.0,0.0
3,u0004,0,124.0,10.0,19.0,105.0,254.0,77.0,133.0,121.0,218.0,6.0,30.0,0.0,0.0
4,u0005,0,48.0,4.0,2.0,46.0,401.0,55.0,177.0,224.0,398.0,0.0,3.0,0.0,0.0


In [15]:
print(wa.shape)
wa.head(3)

(4808343, 8)


,uid,wa_name,visit_cnt,visit_dura,up_flow,down_flow,wa_type,date
0,u0001,155导航,5.0,207.0,313.0,457.0,0.0,14.0
1,u0001,155导航,7.0,396.0,547.0,659.0,0.0,4.0
2,u0001,155导航,10.0,3212.0,781.0,941.0,0.0,12.0


In [16]:
wa_feat = pd.DataFrame()
grouped = wa.groupby('uid')['visit_cnt']
x = grouped.apply(lambda x : x.sum())
wa_feat['uid'] = x.index
wa_feat['wa_visit_cnt_sum'] = x.values

grouped = wa.groupby('uid')['wa_name']
x = grouped.apply(lambda x : len(set(x)))
wa_feat['wa_name_count_unique'] = x.values

grouped = wa.groupby('uid')['visit_dura']
x = grouped.apply(lambda x : x.mean())
wa_feat['wa_visit_dura_mean'] = x.values

x = grouped.apply(lambda x : x.std())
wa_feat['wa_visit_dura_std'] = x.values

grouped = wa.groupby('uid')['up_flow']
x = grouped.apply(lambda x : x.mean())
wa_feat['wa_up_flow_mean'] = x.values

x = grouped.apply(lambda x : x.std())
wa_feat['wa_up_flow_std'] = x.values

grouped = wa.groupby('uid')['down_flow']
x = grouped.apply(lambda x : x.mean())
wa_feat['wa_down_flow_mean'] = x.values

x = grouped.apply(lambda x : x.std())
wa_feat['wa_down_flow_std'] = x.values

grouped = wa.groupby(['uid','wa_type'])['wa_name']
x = grouped.apply(lambda x : x.count())
x = x.unstack(fill_value=0).reset_index(drop=True)
x.columns = ['0','1']
wa_feat['wa_count_type0'] = x['0']
wa_feat['wa_count_type1'] = x['1']

wa_feat.head(5)

,uid,wa_visit_cnt_sum,wa_name_count_unique,wa_visit_dura_mean,wa_visit_dura_std,wa_up_flow_mean,wa_up_flow_std,wa_down_flow_mean,wa_down_flow_std,wa_count_type0,wa_count_type1
0,u0001,97694.0,181,96725.498701,296715.319506,172279.362771,749185.379948,8.325406e+05,4.243115e+06,955.0,200.0
1,u0002,178.0,5,15229.413793,19554.843909,4433.551724,5429.253721,3.267207e+03,3.908229e+03,29.0,0.0
2,u0003,23793.0,178,25471.857755,83464.179838,60843.138817,356624.138405,1.115881e+06,1.445421e+07,982.0,185.0
3,u0004,56631.0,298,40928.322619,128052.491187,181739.734524,932711.435806,2.082341e+06,1.564955e+07,1372.0,308.0
4,u0005,13835.0,130,25504.252730,72239.474882,63241.511700,507181.731745,3.556163e+05,1.617302e+06,557.0,84.0


In [17]:
uid_train = uid_train.merge(wa_feat,on='uid',how='left').reset_index(drop=True)
uid_train = uid_train.fillna(0)
uid_train.head(5)

,uid,label,sms_opp_count_all,sms_opp_count_unique,sms_opp_count_out,sms_opp_count_in,voice_opp_count_all,voice_opp_count_unique,voice_opp_count_out,voice_opp_count_in,...,wa_visit_cnt_sum,wa_name_count_unique,wa_visit_dura_mean,wa_visit_dura_std,wa_up_flow_mean,wa_up_flow_std,wa_down_flow_mean,wa_down_flow_std,wa_count_type0,wa_count_type1
0,u0001,0,95.0,11.0,39.0,56.0,79.0,22.0,33.0,46.0,...,97694.0,181,96725.498701,296715.319506,172279.362771,749185.379948,8.325406e+05,4.243115e+06,955.0,200.0
1,u0002,0,1.0,1.0,0.0,1.0,2.0,2.0,2.0,0.0,...,178.0,5,15229.413793,19554.843909,4433.551724,5429.253721,3.267207e+03,3.908229e+03,29.0,0.0
2,u0003,0,28.0,2.0,0.0,28.0,21.0,15.0,7.0,14.0,...,23793.0,178,25471.857755,83464.179838,60843.138817,356624.138405,1.115881e+06,1.445421e+07,982.0,185.0
3,u0004,0,124.0,10.0,19.0,105.0,254.0,77.0,133.0,121.0,...,56631.0,298,40928.322619,128052.491187,181739.734524,932711.435806,2.082341e+06,1.564955e+07,1372.0,308.0
4,u0005,0,48.0,4.0,2.0,46.0,401.0,55.0,177.0,224.0,...,13835.0,130,25504.252730,72239.474882,63241.511700,507181.731745,3.556163e+05,1.617302e+06,557.0,84.0


In [18]:
uid_train.to_csv('../train_feat.csv')